<a href="https://colab.research.google.com/github/borislevant/SciComPy/blob/master/cifar10_optuna_package.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this notebook we will demonstrate the work on the CIFAR-10 data set from the simplest Softmax model (Logistic regression) to the more advanced methods

## 1. Introduction
The CIFAR-10 dataset contains 60,000 color images of 32 x 32 pixels in 3 channels divided into 10
classes. Each class contains 6,000 images. The training set contains 50,000 images, while the test sets
provides 10,000 images. This image taken from the CIFAR repository ( <a href = "https://www.cs.toronto.edu/~kriz/cifar.html">https://www.cs.toronto.edu/~kriz/cifar.html </a>). This is a classification problem with 10 classes(muti-label classification). We can take a view on this image for more comprehension of the dataset. 

![cifar10.png](https://github.com/borislevant/SciComPy/blob/master/cifar10.png?raw=1)


The challenge is to recognize previously unseen images and assign them to one of the 10 classes.

Ok Let's get started.

## 2. Import and Preprocess the data



### 2.1 Import all required libraries

In [1]:
from __future__ import print_function
from tensorflow import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, InputLayer
from keras.layers import Conv2D, MaxPooling2D
import os

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix, classification_report
import itertools

%matplotlib inline


In [2]:
!pip install optuna
import optuna
from optuna.integration import KerasPruningCallback
from optuna.trial import TrialState

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 5.1 MB/s 
     |████████████████████████████████| 81 kB 4.8 MB/s 
     |████████████████████████████████| 209 kB 42.1 MB/s 
     |████████████████████████████████| 78 kB 8.5 MB/s 
     |████████████████████████████████| 49 kB 4.2 MB/s 
     |████████████████████████████████| 147 kB 55.5 MB/s 
     |████████████████████████████████| 112 kB 24.8 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=ec72a3e1da6a11e105a1e20b5a223243f30302c275c18b7f5197ac11e49a1d45
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


### 2.2 Import and preproces of data 
We load the data and split it between train and test sets


In [3]:
# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


170508288/170498071 [==============================] - 2s 0us/step
x_train shape: (50000, 32, 32, 3)
y_train shape: (50000, 1)
50000 train samples
10000 test samples


In [4]:
# Normalize the data. Before we need to connvert data type to float for computation.
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices. This is called one hot encoding.
num_classes = 10  # Number of class for the dataset
num_epochs = 20
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##3. Softmax model

In [6]:
def create_softmax_model(trial):
    l2_regularization=trial.suggest_float("l2_regularization", 1e-5, 1e-1, log=True)
    reg = keras.regularizers.l2(l2=l2_regularization)

    model = Sequential()
    model.add(InputLayer(input_shape=x_train.shape[1:]))
    model.add(Flatten())
    model.add(Dense(num_classes, activation='softmax', kernel_regularizer=reg))
    model.summary()
    return model

def accuracy_softmax_mode(trial):
    learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    model = create_softmax_model(trial)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    batch_size = trial.suggest_int("batch_size", 2, 256, log=True)
    history = model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=num_epochs,
                validation_data=(x_test, y_test),
                shuffle=True)

    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]


##3.1 Optimizing the Softmax model

In [7]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(accuracy_softmax_mode, n_trials=20)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))



[I 2022-07-21 06:19:59,008] A new study created in memory with name: no-name-f563b791-124b-4bfd-a491-d32a9a024347


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 3072)              0         
                                                                 
 dense (Dense)               (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
313/313 [==============================] - 5s 5ms/step - loss: 2.9121 - accuracy: 0.2827 - val_loss: 2.0883 - val_accuracy: 0.3174
Epoch 2/20
313/313 [==============================] - 1s 4ms/step - loss: 2.2608 - accuracy: 0.3067 - val_loss: 2.0902 - val_accuracy: 0.3165
Epoch 3/20
313/313 [==============================] - 1s 4ms/step - loss: 2.2046 - accuracy: 0.3190 - val_loss: 2.0124 - val_accuracy: 0.3357
Epoch 4/20
313/313 [==

[I 2022-07-21 06:20:33,678] Trial 0 finished with value: 0.3707999885082245 and parameters: {'learning_rate': 0.008796923289167547, 'l2_regularization': 0.0002295533555353467, 'batch_size': 160}. Best is trial 0 with value: 0.3707999885082245.


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_1 (Dense)             (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1316/1316 [==============================] - 4s 3ms/step - loss: 2.0090 - accuracy: 0.2881 - val_loss: 1.9066 - val_accuracy: 0.3315
Epoch 2/20
1316/1316 [==============================] - 4s 3ms/step - loss: 1.8680 - accuracy: 0.3472 - val_loss: 1.8478 - val_accuracy: 0.3555
Epoch 3/20
1316/1316 [==============================] - 4s 3ms/step - loss: 1.8305 - accuracy: 0.3666 - val_loss: 1.8179 - val_accuracy: 0.3663
Epoch 4/20
131

[I 2022-07-21 06:21:55,507] Trial 1 finished with value: 0.39320001006126404 and parameters: {'learning_rate': 8.147597797408284e-05, 'l2_regularization': 0.00019598099775838747, 'batch_size': 38}. Best is trial 1 with value: 0.39320001006126404.


...

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_18 (Flatten)        (None, 3072)              0         
                                                                 
 dense_18 (Dense)            (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1191/1191 [==============================] - 4s 3ms/step - loss: 2.0452 - accuracy: 0.2685 - val_loss: 1.9273 - val_accuracy: 0.3303
Epoch 2/20
1191/1191 [==============================] - 4s 3ms/step - loss: 1.8940 - accuracy: 0.3392 - val_loss: 1.8674 - val_accuracy: 0.3548
Epoch 3/20
1191/1191 [==============================] - 4s 3ms/step - loss: 1.8516 - accuracy: 0.3562 - val_loss: 1.8366 - val_accuracy: 0.3628
Epoch 4/20
11

[I 2022-07-21 07:21:44,356] Trial 18 finished with value: 0.396699994802475 and parameters: {'learning_rate': 5.612540938668213e-05, 'l2_regularization': 3.458436186709661e-05, 'batch_size': 42}. Best is trial 18 with value: 0.396699994802475.


Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_19 (Flatten)        (None, 3072)              0         
                                                                 
 dense_19 (Dense)            (None, 10)                30730     
                                                                 
Total params: 30,730
Trainable params: 30,730
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
8334/8334 [==============================] - 26s 3ms/step - loss: 1.9724 - accuracy: 0.2962 - val_loss: 1.8685 - val_accuracy: 0.3459
Epoch 2/20
8334/8334 [==============================] - 23s 3ms/step - loss: 1.8471 - accuracy: 0.3566 - val_loss: 1.8274 - val_accuracy: 0.3660
Epoch 3/20
8334/8334 [==============================] - 23s 3ms/step - loss: 1.8141 - accuracy: 0.3706 - val_loss: 1.8078 - val_accuracy: 0.3670
Epoch 4/20

[I 2022-07-21 07:29:38,812] Trial 19 finished with value: 0.3939000070095062 and parameters: {'learning_rate': 4.1972047203366254e-05, 'l2_regularization': 3.851240270424532e-05, 'batch_size': 6}. Best is trial 18 with value: 0.396699994802475.


Study statistics: 
  Number of finished trials:  20
  Number of pruned trials:  0
  Number of complete trials:  20
Best trial:
  Value:  0.396699994802475
  Params: 
    learning_rate: 5.612540938668213e-05
    l2_regularization: 3.458436186709661e-05
    batch_size: 42


#4. Fully connectd Neural Network

In [7]:
def create_fnn_model(trial):
    l2_regularization=trial.suggest_float("l2_regularization", 1e-5, 1e-1, log=True)

    model = Sequential()
    model.add(InputLayer(input_shape=x_train.shape[1:]))
    model.add(Flatten())

    n_layers = trial.suggest_int("n_layers", 1, 4)
    for i in range(n_layers):
            num_hidden_exp = trial.suggest_int("n_units_l{}".format(i), 4, 8)
            num_hidden = int(np.pow(2, num_hidden_exp))
            model.add(Dense(num_hidden, activation="relu"))
            dropout = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
            model.add(Dropout(rate=dropout))
    model.add(Dense(num_classes, activation='softmax'))
    model.summary()
    return model

def accuracy_fnn_mode(trial):
    learning_rate=trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    opt = keras.optimizers.Adam(learning_rate=learning_rate)
    model = create_fnn_model(trial)
    model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

    batch_size_exp = trial.suggest_int("batch_size", 1, 8)
    batch_size = int(np.pow(2, batch_size_exp))
    history = model.fit(x_train, y_train,
                batch_size=batch_size,
                epochs=num_epochs,
                validation_data=(x_test, y_test),
                shuffle=True)

    score = model.evaluate(x_test, y_test, verbose=0)
    return score[1]


In [8]:
study = optuna.create_study(direction="maximize", pruner=optuna.pruners.MedianPruner())
study.optimize(accuracy_fnn_mode, n_trials=100)
pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

print("Study statistics: ")
print("  Number of finished trials: ", len(study.trials))
print("  Number of pruned trials: ", len(pruned_trials))
print("  Number of complete trials: ", len(complete_trials))

print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


[I 2022-07-21 16:35:26,761] A new study created in memory with name: no-name-182775c5-9a76-4376-ab01-570f724ac9ea


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 3072)              0         
                                                                 
 dense_3 (Dense)             (None, 37)                113701    
                                                                 
 dropout_2 (Dropout)         (None, 37)                0         
                                                                 
 dense_4 (Dense)             (None, 253)               9614      
                                                                 
 dropout_3 (Dropout)         (None, 253)               0         
                                                                 
 dense_5 (Dense)             (None, 27)                6858      
                                                                 
 dropout_4 (Dropout)         (None, 27)               

KeyboardInterrupt: ignored